**Install python dependencies**

In [1]:
!pip install -q -r ./requirements.txt

**Load python libraries**

In [2]:
from sqlalchemy import create_engine
from os import getenv
import pandas as pd

**Connect to database**

In [3]:
db_name = getenv("POSTGRES_DB")
db_user = getenv("POSTGRES_USER")
db_pass = getenv("POSTGRES_PASSWORD")
db_host = 'localhost'
db_port = '5432'
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

**Preprocess Data: Event Data**

In [5]:
# getting event data from database
db_response = db.execute(
    f"""select  (coalesce(p.first_name, '') || ' ' ||coalesce(p.last_name, '')) as name,
       position,
       number as matchday,
       type as event_type,
       count(*) from events
        inner join matchdays m on events.matchday_id = m.id
        inner join players p on events.player_id = p.id
    where corrected is false
    group by name, position, matchday, event_type
    order by name, matchday,  event_type
    """
)

df_event = pd.DataFrame(db_response.fetchall())
df_event.columns = db_response.keys()

df_event

,name,position,matchday,event_type,count
0,Aarón,defender,3,interception,1
1,Aarón,defender,3,pass,10
2,Aarón,defender,3,playerOn,1
3,Aarón,defender,3,successfulTackle,1
4,Aarón,defender,3,throwIn,2
...,...,...,...,...,...
72211,Yuya Osako,attacker,34,shotAtGoal,1
72212,Yuya Osako,attacker,34,successfulTackle,2
72213,Yuya Osako,attacker,34,superPass,1
72214,Yuya Osako,attacker,34,unsuccessfulPass,3


In [8]:
# load event_types
event_types = ['pass', 'unsuccessfulPass', 'superPass', 'throwIn', 'goalAssist', 'farCorner', 'cross', 'freeKick', 'interception', 'goalMissedFar', 'shotAtGoal', 'unsuccessfulPenalty', 'missedChance', 'goal', 'doublePack', 'hattrick', 'penaltyGoal', 'ownGoal', 'foul','awardedPenalty', 'causedPenalty', 'yellowCard', 'secondYellowCard', 'redCard', 'successfulTackle', 'unsuccessfulTackle', 'blockedGoalShot', 'offside', 'error', 'savedPenalty', 'lostPenalty', 'defended', 'goalAgainst',]

33

In [13]:
df_players = df_event.groupby('name')

for player_tuple in df_players:
    player_name = player_tuple[0]
    position = player_tuple[1]['position'].max()

    df_player = player_tuple[1] 

    df_player_type = df_player.groupby('event_type')

    for player_type_tuple in df_player_type:
        print(player_type_tuple[0])
        print(player_type_tuple[1])

    """ for event_type in event_types:


    for matchday in range(1,34):
        if df_player.loc[df_player['matchday'] == matchday].empty == True:
            for event_type in event_types:
                df_player = df_player.append({
                    'name': player_name, 
                    'position': position,
                    'matchday': matchday,
                    'event_type': event_type,
                    'count': 0
                }, ignore_index=True)
        
        if df_player.loc[df_player['matchday'] == matchday].empty == False: """


